<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/07_linear_regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ケーススタディ: solubility data

* 下記の書籍のSection 6.1で紹介されているデータセットを使ってみる。
  * Max Kuhn and Kjell Johnson. Applied Predictive Modeling. Springer, 2013.
  * http://appliedpredictivemodeling.com/data

* 説明変数は下記の228個
  * **Two hundred and eight** binary “fingerprints” that indicate the presence or absence of a particular chemical substructure.
  * **Sixteen** count descriptors, such as the number of bonds or the number of bromine atoms.
  * **Four** continuous descriptors, such as molecular weight or surface area.

* 目的変数はlog solubility
  * 範囲は-11.6から1.6、平均は-2.7

## データを読み込む

In [ ]:
import os
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

%config InlineBackend.figure_format = 'retina'

In [ ]:
data_path = '/content/drive/MyDrive/data/'

# 訓練データ
X = pd.read_csv(os.path.join(data_path, 'solTrainX.csv'))
y = pd.read_csv(os.path.join(data_path, 'solTrainY.csv'))['x']

# テストデータ
X_test = pd.read_csv(os.path.join(data_path, 'solTestX.csv'))
y_test = pd.read_csv(os.path.join(data_path, 'solTestY.csv'))['x']

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
y.head()

## 訓練データのEDA

### 連続値をとる特徴量

In [ ]:
continuous = [s for s in X.columns if s[:3] in ['Num', 'Hyd', 'Mol', 'Sur']]
print('There are', len(continuous), 'continuous features.')
print(continuous)

* 目的変数との関連を可視化する。

In [ ]:
# 図を描くために、一時的にこうしておく
X['solubility'] = y

sns.set_style("whitegrid")
fig = plt.figure(figsize=(21, 18))
for i in range(20):
  ax = fig.add_subplot(5, 4, i+1)
  sns.regplot(x=continuous[i], y='solubility', data=X, ax=ax)
  if i % 4 != 0:
    ax.set_ylabel('')

# 図を描き終えたので、元に戻す
X = X.drop('solubility', axis=1)

### 0/1の2値をとる特徴量

In [ ]:
binary = X.columns[X.columns.str.startswith('FP')]
print(len(binary), 'binary features')
X[binary].describe()

## そのまま線形回帰を適用

* 検証データを取り出しておく

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

* 線形回帰モデルのパラメータを最小二乗法で推定させる


In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

* 検証データでの予測を評価する
  * 評価尺度はRMSE (root-mean-square error) とする。
  * RMSEは、小さいほど、良い。


In [ ]:
y_valid_pred = reg.predict(X_valid)
print(f'RMSE: {root_mean_squared_error(y_valid, y_valid_pred):.4f}')

* このRMSEを改善できるかどうか、いろいろ試行錯誤する。

## 2値の部分だけをPCAで次元圧縮

In [ ]:
from sklearn.decomposition import PCA

n_components = 100 # 使用する主成分の個数
pca = PCA(n_components=n_components, random_state=123)
pca.fit(X_train[binary])

In [ ]:
plt.plot(np.arange(1, pca.n_components_ + 1), np.cumsum(pca.explained_variance_ratio_));

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
X_train_binary_embedded = pca.transform(X_train[binary])

In [ ]:
X_train_binary_embedded.shape

In [ ]:
X_train_binary_embedded[0]

* 2値変数の部分を次元削減した後のものと、元の連続値の部分とを、くっつける


In [ ]:
X_train_embedded = np.concatenate([X_train_binary_embedded, X_train[continuous]], 1)

* X_train_embeddedを使って線形回帰モデルのパラメータを決める
* そして検証データでの予測スコアを求める
  * もちろん、検証データの2値の部分も、次元削減する必要があります。


In [ ]:
reg = LinearRegression()
reg.fit(X_train_embedded, y_train)

X_valid_binary_embedded = pca.transform(X_valid[binary])
X_valid_embedded = np.concatenate([X_valid_binary_embedded, X_valid[continuous]], 1)
y_valid_pred = reg.predict(X_valid_embedded)
print(f'RMSE: {root_mean_squared_error(y_valid, y_valid_pred):.4f}')

### 演習問題
* 以下のコードで、`n_components=0.99`の部分は、どういう意味だろうか？

In [ ]:
pca = PCA(n_components=0.99, random_state=123)
X_train_binary_embedded = pca.fit_transform(X_train[binary])

X_train_embedded = np.concatenate([X_train_binary_embedded, X_train[continuous]], 1)

reg = LinearRegression()
reg.fit(X_train_embedded, y_train)

X_valid_binary_embedded = pca.transform(X_valid[binary])
X_valid_embedded = np.concatenate([X_valid_binary_embedded, X_valid[continuous]], 1)
y_valid_pred = reg.predict(X_valid_embedded)
print(f'RMSE: {root_mean_squared_error(y_valid, y_valid_pred):.4f}')

## 2値変数のインタラクションを考慮
* PolynomialFeaturesを2次の設定で使う
* その上で主成分分析を適用

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
poly.fit(X_train[binary])

In [ ]:
X_train_binary_poly = poly.transform(X_train[binary])

In [ ]:
pca = PCA(n_components=0.99, random_state=123)
pca.fit(X_train_binary_poly)
plt.plot(np.arange(1, pca.n_components_ + 1), np.cumsum(pca.explained_variance_ratio_));

* 2次のインタラクションを含めた2値変数の部分を次元削減した後のものと、元の連続値の部分とを、くっつける


In [ ]:
X_train_embedded = np.concatenate([pca.transform(X_train_binary_poly),
                                   X_train[continuous]], 1)

In [ ]:
reg = LinearRegression()
reg.fit(X_train_embedded, y_train)

X_valid_binary_poly_embedded = pca.transform(poly.transform(X_valid[binary]))
X_valid_embedded = np.concatenate([X_valid_binary_poly_embedded, X_valid[continuous]], 1)
y_valid_pred = reg.predict(X_valid_embedded)
print(f'RMSE: {root_mean_squared_error(y_valid, y_valid_pred):.4f}')

## Ridge回帰
* alphaをチューニングする。

In [ ]:
from sklearn.linear_model import Ridge

for alpha in 10. ** np.arange(-5, 6):
  reg = Ridge(alpha=alpha)
  reg.fit(X_train_embedded, y_train)
  y_valid_pred = reg.predict(X_valid_embedded)
  print(f'alpha: {alpha:.1e}; RMSE: {root_mean_squared_error(y_valid, y_valid_pred):.4f}')

## 交差検証

* ここまで使っていた訓練データと検証データを一つにまとめ直す。

In [ ]:
X_train_valid = pd.concat([X_train, X_valid], axis=0)
y_train_valid = pd.concat([y_train, y_valid], axis=0)

* 5-fold交差検証をおこなう

In [ ]:
from sklearn.model_selection import KFold

poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)

n_components = 400
pca = PCA(n_components=n_components, random_state=123)

kf = KFold(n_splits=5, shuffle=True, random_state=123)

errors = []
for train_index, valid_index in kf.split(X_train_valid):
  X_train, X_valid = X_train_valid.iloc[train_index], X_train_valid.iloc[valid_index]
  y_train, y_valid = y_train_valid.iloc[train_index], y_train_valid.iloc[valid_index]

  X_train_binary_embedded = pca.fit_transform(poly.fit_transform(X_train[binary]))
  X_train_embedded = np.concatenate([X_train_binary_embedded, X_train[continuous]], 1)

  X_valid_binary_embedded = pca.transform(poly.transform(X_valid[binary]))
  X_valid_embedded = np.concatenate([X_valid_binary_embedded, X_valid[continuous]], 1)

  split_errors = []
  for alpha in 10. ** np.arange(-5, 6):
    reg = Ridge(alpha=alpha)
    reg.fit(X_train_embedded, y_train)
    y_valid_pred = reg.predict(X_valid_embedded)
    error = root_mean_squared_error(y_valid, y_valid_pred)
    print(f'alpha: {alpha:.1e}; RMSE: {error:.4f}', flush=True)
    split_errors.append(error)
  errors.append(split_errors)
  print('-' * 80)

errors = np.array(errors)
print(errors)

* 異なるfoldでRMSEがかなり違う。
 * もっと前から交差検証をしておくべきだったのかも。

In [ ]:
for alpha, mean_rmse in zip(10. ** np.arange(-5, 6), errors.mean(0)):
  print(f'alpha: {alpha:.1e}; mean RMSE: {mean_rmse:.4f}')

## 一番良かった手法で最終評価

* ここまででは、以下の設定にたどり着いた。
 * 2値変数の2次のインタラクションを使う。
 * 2値の部分をPCAで400次元に落とす。
 * Ridge回帰を`alpha=10.0`で適用
* そこで、この手法を採用して、テストデータで最終評価する。


In [ ]:
poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)

n_components = 400
pca = PCA(n_components=n_components, random_state=123)

X_train_valid_binary_poly_embedded = pca.fit_transform(poly.fit_transform(X_train_valid[binary]))
X_train_valid_embedded = np.concatenate([X_train_valid_binary_poly_embedded, X_train_valid[continuous]], 1)

reg = Ridge(alpha=10.0)
reg.fit(X_train_valid_embedded, y_train_valid)

X_test_binary_poly_embedded = pca.transform(poly.transform(X_test[binary]))
X_test_embedded = np.concatenate([X_test_binary_poly_embedded, X_test[continuous]], 1)
y_test_pred = reg.predict(X_test_embedded)
print(f'RMSE: {root_mean_squared_error(y_test, y_test_pred):.4f}')

# 課題
* solubilityデータセットの、上で作った検証データに対して、できるだけ予測性能の良いモデルを見つけよう
  * Ridge回帰やLassoを使ってもいいです。
  * 特徴量はどのように加工してもいいです。（上では2値変数にPCAを使った。）
* 検証データを使って見つけた最も良いモデルを、最後に一回、テストデータで評価してみよう